# White-box Targeted Attack on CIFAR10

## Load model and data

In [1]:
import torch
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

images, labels = load_cifar10(n_examples=5)
print('[Data loaded]')

device = 'cuda'
model = load_model('Standard', norm='Linf').to(device)
acc = clean_accuracy(model, images.to(device), labels.to(device))
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))

100%|███████████████████████| 170498071/170498071 [00:09<00:00, 18770616.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
[Data loaded]


Downloading...
From (original): https://drive.google.com/uc?id=1t98aEuzeTL8P7Kpd5DIrCoCL21BNZUhC
From (redirected): https://drive.google.com/uc?id=1t98aEuzeTL8P7Kpd5DIrCoCL21BNZUhC&confirm=t&uuid=5855c01e-3ded-406d-86b7-36b7690d39e1
To: /home/mwm/repositories/pwn/adwersarialny_artykul/adversarial-attacks-pytorch/demo/models/cifar10/Linf/Standard.pt
100%|████████████████████████████████████████| 292M/292M [00:15<00:00, 19.4MB/s]


[Model loaded]
Acc: 100.00 %


## Adversarial Attack

In [2]:
from torchattacks import PGD
atk = PGD(model, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk.set_mode_targeted_by_label()
print(atk)

Attack mode is changed to 'targeted(label)'.
PGD(model_name=WideResNet, device=cuda:0, attack_mode=targeted(label), targeted=True, normalization_used=False, eps=0.03137254901960784, alpha=0.008888888888888889, steps=10, random_start=True)


In [3]:
print(labels)
new_labels = (labels + 1) % 10
print(new_labels)

tensor([3, 8, 8, 0, 6])
tensor([4, 9, 9, 1, 7])


In [4]:
adv_images = atk(images, new_labels)
adv_pred = model(adv_images)
print(labels)
print(torch.argmax(adv_pred, 1))

targeted(label)
tensor([3, 8, 8, 0, 6])
tensor([4, 9, 9, 1, 7], device='cuda:0')
